In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [2]:
# Create reference to CSV file
csv_path = "Resources/purchase_data.csv"

# Import the CSV into a pandas DataFrame
p_data = pd.read_csv(csv_path, low_memory=False)
p_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Get Player COunt
player_count = len(p_data['SN'].unique())
print(f"Player Count: {player_count}")

Player Count: 576


In [4]:
#Purchasing Analysis (Total)
unique_items = len(p_data['Item Name'].unique())
average_pr_price = '${:,.2f}'.format(p_data['Price'].mean())
purchase_count = len(p_data)
total_revenue = '${:,.2f}'.format(p_data['Price'].sum())


print(f"Unique Items: {unique_items}")
print(f"Average Purchase Price: {average_pr_price}")
print(f"Total Purchases: {purchase_count}")
print(f"Total Purchases: {total_revenue}")

Unique Items: 179
Average Purchase Price: $3.05
Total Purchases: 780
Total Purchases: $2,379.77


In [5]:
#Gender Demographics

grouped= p_data.groupby('Gender').SN.nunique()
grouped.head()
grouped= pd.DataFrame(grouped)
grouped = grouped.rename(columns={'SN': 'Count'})
grouped['Percentage'] = grouped['Count'].astype('float')/grouped['Count'].sum()*100
grouped

,Count,Percentage
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722


In [6]:
#Purchasing Analysis
pA = p_data.groupby("Gender").agg({"Price": np.sum, "SN": pd.Series.nunique})
pA = pA.rename(columns={"SN": 'Purchase Count', "Price": "Total Purchase Value"})
pA['Average Purchase Price']= pA['Total Purchase Value']/pA['Purchase Count']
pA

,Total Purchase Value,Purchase Count,Average Purchase Price
Gender,,,
Female,361.94,81,4.468395
Male,1967.64,484,4.065372
Other / Non-Disclosed,50.19,11,4.562727


In [7]:
# Create bins 
bins=[0, 9, 14,19,24,29,34,39,100]
# Create labels for these bins
gnames=['< 10 Years Old','10 - 14', '15-19','20-24','25-29','30-34','35-39','40 and Above']
pd.cut(p_data["Age"], bins, labels=gnames).head()

# Create a GroupBy object based upon "View Group"
aged=p_data.groupby("Age")
# Slice the data and place it into bins
p_data['Age Group'] = pd.cut(p_data["Age"], bins, labels=gnames)
p_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40 and Above
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [8]:
ageGroup= p_data.groupby('Age Group')

groupings = pd.DataFrame(p_data['Age Group'].value_counts())
groupings = groupings.rename(columns={"Age Group": 'Purchase Count'})
groupings['Total Purchase Value'] = ageGroup['Price'].sum()
groupings['Average Purchase Price'] = ageGroup['Price'].sum()/groupings['Purchase Count']

groupings

,Purchase Count,Total Purchase Value,Average Purchase Price
20-24,365,1114.06,3.052219
15-19,136,412.89,3.035956
25-29,101,293.00,2.900990
30-34,73,214.00,2.931507
35-39,41,147.67,3.601707
10 - 14,28,82.78,2.956429
< 10 Years Old,23,77.13,3.353478
40 and Above,13,38.24,2.941538


In [9]:
#TOP 5 SPENDERS

spenders_group= p_data.groupby('SN')
spenders_group_df=spenders_group.count()
spenders_group_df.drop(columns={'Purchase ID','Age','Age Group','Item ID','Price', 'Item Name'}, inplace=True)
spenders_group_df.rename(columns={'Gender': 'Purchase Count'}, inplace= True)
spenders_group['Price'].sum()
spenders_group_df['Total Purchase Value'] =spenders_group['Price'].sum()
spenders_group_df['Average Purchase Price']=spenders_group_df['Total Purchase Value']/spenders_group_df['Purchase Count']
spenders_group_df.sort_values(by=['Purchase Count'], ascending=False).head(5)

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,18.96,3.792000
Iral74,4,13.62,3.405000
Idastidru52,4,15.45,3.862500
Asur53,3,7.44,2.480000
Inguron55,3,11.11,3.703333


In [28]:
#Most Popular Items
pop_items= p_data.groupby('Item Name')
# pop_count =pop_items.count().sort_values(by="Item ID", ascending=False).head(5)
pop_count =pop_items.count()

pop_sum =pop_items.sum()
pop_merge = pd.merge(pop_sum, pop_count, on="Item Name")
pop_merge.drop(columns={'Purchase ID_x','Age_x','Purchase ID_y','SN','Age_y','Gender','Item ID_y','Price_y'}, inplace=True)
pop_merge.rename(columns={"Item ID_x": 'Item ID', 'Price_x': 'Total Purchase Value','Age Group': 'Purchase Count'}, inplace=True)

pop_merge['Item ID']=(pop_merge['Item ID']/pop_merge['Purchase Count']).astype('int')

pop_merge['Item Pric']

,Item ID,Total Purchase Value,Purchase Count
Item Name,,,
Abyssal Shard,162,13.35,5
"Aetherius, Boon of the Blessed",137,16.95,5
Agatha,120,18.48,6
Alpha,130,6.21,3
"Alpha, Oath of Zeal",79,12.15,3
"Alpha, Reach of Ending Hope",47,3.58,1
Amnesia,22,13.08,6
Apocalyptic Battlescythe,93,11.82,6
Arcane Gem,84,11.37,3
